In [1]:
import pandas as pd 
def load_train_data():
    data = pd.read_csv('Acoustic_train.csv')
    return data

def load_test_data():
    data = pd.read_csv('Acoustic_test_no_class.csv')
    return data

train_data = load_train_data()
test_data = load_test_data()

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


In [3]:

# Load and preprocess the data
X_train = train_data.drop(columns=["Index", "Class"])
y_train = train_data["Class"]


In [4]:

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Split into train and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42
)


In [5]:
# Define a more complex Autoencoder Architecture
input_dim = X_train_scaled.shape[1]
encoding_dim = 128  # Increased dimension of the latent space

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
encoded = Dense(64, activation='tanh')(encoded)
decoded = Dense(encoding_dim, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Build the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder with increased epochs
autoencoder.fit(X_train_split, X_train_split,  # Input equals output
                epochs=150, batch_size=64, validation_data=(X_val, X_val), verbose=1)


Epoch 1/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 1.2221 - val_loss: 1.2450
Epoch 2/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1804 - val_loss: 1.2035
Epoch 3/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1445 - val_loss: 1.1582
Epoch 4/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0873 - val_loss: 1.1098
Epoch 5/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0912 - val_loss: 1.0650
Epoch 6/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.0051 - val_loss: 1.0316
Epoch 7/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9639 - val_loss: 1.0118
Epoch 8/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9870 - val_loss: 0.9992
Epoch 9/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9595 - val_loss: 0.9878
Epoch 10/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.9170 - val_loss: 0.9751
Epoch 11/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9222 - val_loss: 0.9632
Epoch 12/150
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9237 - val_loss: 0

In [6]:

# Extract Encoder Model
encoder = Model(inputs=input_layer, outputs=encoded)


In [7]:

# Get Encoded Features
X_train_encoded = encoder.predict(X_train_scaled)
X_val_encoded = encoder.predict(X_val)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [8]:
from sklearn.svm import SVC

# Encode the split training data
X_train_split_encoded = encoder.predict(X_train_split)

# Train an SVM Classifier
clf = SVC(kernel='rbf', random_state=42)
clf.fit(X_train_split_encoded, y_train_split)

# Make predictions
y_val_pred = clf.predict(X_val_encoded)

# Calculate Accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy with Autoencoder Features (SVM):", accuracy)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step
Validation Accuracy with Autoencoder Features (SVM): 0.8166666666666667


In [9]:
# Preprocess the test data
X_test = test_data.drop(columns=["Index"])
X_test_scaled = scaler.transform(X_test)

# Encode the test data
X_test_encoded = encoder.predict(X_test_scaled)

# Predict the classes using the trained SVM classifier
y_test_pred = clf.predict(X_test_encoded)

print("Predicted Classes for Test Data:", y_test_pred)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Predicted Classes for Test Data: ['sad' 'relax' 'relax' 'sad' 'relax' 'relax' 'sad' 'sad' 'sad' 'happy'
 'sad' 'happy' 'sad' 'relax' 'happy' 'relax' 'happy' 'angry' 'angry'
 'angry' 'sad' 'angry' 'happy' 'sad' 'sad' 'angry' 'happy' 'sad' 'angry'
 'happy' 'angry' 'happy' 'sad' 'relax' 'sad' 'angry' 'happy' 'angry'
 'happy' 'happy' 'angry' 'sad' 'relax' 'sad' 'relax' 'angry' 'happy'
 'happy' 'relax' 'happy' 'relax' 'happy' 'happy' 'relax' 'angry' 'relax'
 'relax' 'happy' 'relax' 'happy' 'happy' 'happy' 'happy' 'relax' 'happy'
 'sad' 'happy' 'sad' 'happy' 'relax' 'relax' 'happy' 'sad' 'sad' 'angry'
 'sad' 'happy' 'angry' 'happy' 'sad' 'relax' 'sad' 'relax' 'sad' 'sad'
 'relax' 'angry' 'angry' 'happy' 'relax' 'relax' 'angry' 'happy' 'sad'
 'happy' 'angry' 'angry' 'happy' 'happy' 'relax']
